In [1]:
import warnings
warnings.filterwarnings('ignore')

import shutil
import pytorch_lightning as pl
import torch
import yaml
import os
import pickle
import logging
import sys
pkg_path = "/nfs/homedirs/nikoghos/project-1/"
if pkg_path not in sys.path:
    sys.path.append(pkg_path)
    
from audio_classification.tools import *
import optuna
from pytorch_lightning import loggers as pl_loggers
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
import yaml
with open("../audio_classification/configs/crnn_urbansound8k.yaml", "r") as config_file:
    cfg = yaml.load(config_file)

In [3]:
cfg

{'MODEL': {'NAME': 'LitCRNN',
  'NUM_CLASSES': 10,
  'CRNN': {'MIXUP': True, 'INCLUDE_TOP': True}},
 'DATASET': {'NAME': 'UrbanSounds8K',
  'ANNOTATION_PATH': '/nfs/students/winter-term-2020/project-1/datasets/UrbanSound8K/metadata/UrbanSound8K.csv',
  'FILE_PATH': '/nfs/students/winter-term-2020/project-1/datasets/UrbanSound8K/audio/',
  'VAL_FOLD': 5,
  'WEIGHT': 'NORMAL'},
 'DATALOADER': {'BATCH_SIZE': 6, 'NUM_WORKERS': 20},
 'SOLVER': {'LOG_PATH': '../logs/',
  'NUM_GPUS': 1,
  'MAX_EPOCH': 35,
  'MIN_EPOCH': 10,
  'LEARNING_RATE': 0.0004881731392294949,
  'WEIGHT_DECAY': 2.628328526113425e-08,
  'STEP_SIZE': 7,
  'GAMMA': 0.1,
  'ALPHA': 0.7608399210841633},
 'CHECKPOINT': {'SAVE_NAME': 'crnn-us8k',
  'SAVE_PATH': '../weights',
  'SAVE_TOP_K': 1}}

In [4]:
from pytorch_lightning import Callback

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [5]:
def save_model(model, file_name, directory = "models"):
    model = model.cpu()
    model_dict = {cfg["MODEL"]["NAME"]:{"state_dict":model.state_dict(), "hparams": model.hparams}}
    if not os.path.exists(directory):
        os.makedirs(directory)
    pickle.dump(model_dict, open(os.path.join(directory, file_name), 'wb', 4))

In [6]:
def objective(trial):
    # Use this callback to collect the validation accuracies
        
    metrics_callback = MetricsCallback()
    
    trial_hparams = {"batch_size": 6, 
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-8, 1e-5),
                     "weight_decay":  trial.suggest_loguniform("weight_decay", 1e-8, 1e-5),
                     "alpha": trial.suggest_loguniform("alpha", 0.01, 2.5)
                    }

    train_loader, val_loader, test_loader, class_weights = get_dataloader(cfg, trial_hparams, transform=get_transform(cfg))
    
    if class_weights is not None:
        class_weights = torch.tensor(class_weights).to(device=torch.device('cuda'))
    
    early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_acc")
    trainer = pl.Trainer(gpus=cfg["SOLVER"]["NUM_GPUS"],
                         min_epochs=cfg["SOLVER"]["MIN_EPOCH"],
                         max_epochs=37,
                         progress_bar_refresh_rate=10,
                         callbacks=[metrics_callback, early_stop_callback],
                         logger=True,
                        )
    
    model = get_model(cfg, class_weights, trial_hparams, train_loader, val_loader)
    
    model.prepare_data()
    trainer.fit(model)

    save_model(model, '{}.p'.format(trial.number), "checkpoints")

    # return validation accuracy from latest model, as that's what we want to maximize by our hyper param search
    return metrics_callback.metrics[-1]["val_acc"]

In [7]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=25, timeout=86000)

[I 2020-12-17 13:39:17,982] A new study created in memory with name: no-name-91191b5d-58fa-47fa-b17f-d55f965dc3d9
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | conv1          | Conv2d      | 640   
5  | bn1            | BatchNorm2d | 128   
6  | dropout1       | Dropout     | 0     
7  | conv2          | Conv2d      | 73.9 K
8  | bn2            | BatchNorm2d | 256   
9  | dropout2       | Dropout     | 0     
10 | conv3          | Conv2d      | 147 K 
11 | bn3            | BatchNorm2d | 256   
12 | dropout3       | Dropout     | 0     
13 | conv4          | Conv2d      | 147 K 
14 | bn4            | BatchNorm2d | 256   
15 | dr

[I 2020-12-17 15:06:50,978] Trial 0 finished with value: 0.48493337631225586 and parameters: {'learning_rate': 6.259710268555027e-06, 'weight_decay': 8.802969581182646e-07, 'alpha': 0.14393975649145063}. Best is trial 0 with value: 0.48493337631225586.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | conv1          | Conv2d      | 640   
5  | bn1            | BatchNorm2d | 128   
6  | dropout1       | Dropout     | 0     
7  | conv2          | Conv2d      | 73.9 K
8  | bn2            | BatchNorm2d | 256   
9  | dropout2       | Dropout     | 0     
10 | conv3          | Conv2d      | 147 K 
11 | bn3            | BatchNorm2d | 256   
12 | dropout3       | Dropout     | 0     
13 | conv4          | Conv2d      | 147 K 
14 | bn4            | BatchNorm2d | 256   
15 | dropout4       | Dropout     | 0     
16 | rnn            | GRU         | 21.9 K
17 | dropout_rnn    | Dropout     |

[I 2020-12-17 16:34:44,134] Trial 1 finished with value: 0.19984450936317444 and parameters: {'learning_rate': 1.356344343688718e-07, 'weight_decay': 1.4626393588298454e-08, 'alpha': 0.8878387662091798}. Best is trial 0 with value: 0.48493337631225586.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | conv1          | Conv2d      | 640   
5  | bn1            | BatchNorm2d | 128   
6  | dropout1       | Dropout     | 0     
7  | conv2          | Conv2d      | 73.9 K
8  | bn2            | BatchNorm2d | 256   
9  | dropout2       | Dropout     | 0     
10 | conv3          | Conv2d      | 147 K 
11 | bn3            | BatchNorm2d | 256   
12 | dropout3       | Dropout     | 0     
13 | conv4          | Conv2d      | 147 K 
14 | bn4            | BatchNorm2d | 256   
15 | dropout4       | Dropout     | 0     
16 | rnn            | GRU         | 21.9 K
17 | dropout_rnn    | Dropout     |

...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | conv1          | Conv2d      | 640   
5  | bn1            | BatchNorm2d | 128   
6  | dropout1       | Dropout     | 0     
7  | conv2          | Conv2d      | 73.9 K
8  | bn2            | BatchNorm2d | 256   
9  | dropout2       | Dropout     | 0     
10 | conv3          | Conv2d      | 147 K 
11 | bn3            | BatchNorm2d | 256   
12 | dropout3       | Dropout     | 0     
13 | conv4          | Conv2d      | 147 K 
14 | bn4            | BatchNorm2d | 256   
15 | dropout4       | Dropout     | 0     
16 | rnn            | GRU         | 21.9 K
17 | dropout_rnn    | Dropout     |

[I 2020-12-18 11:00:30,781] Trial 14 finished with value: 0.6112884283065796 and parameters: {'learning_rate': 9.382846086052564e-06, 'weight_decay': 2.3975765848796826e-06, 'alpha': 0.07669590914597525}. Best is trial 10 with value: 0.6708158850669861.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | conv1          | Conv2d      | 640   
5  | bn1            | BatchNorm2d | 128   
6  | dropout1       | Dropout     | 0     
7  | conv2          | Conv2d      | 73.9 K
8  | bn2            | BatchNorm2d | 256   
9  | dropout2       | Dropout     | 0     
10 | conv3          | Conv2d      | 147 K 
11 | bn3            | BatchNorm2d | 256

[I 2020-12-18 12:24:37,428] Trial 15 finished with value: 0.06710060685873032 and parameters: {'learning_rate': 1.1695360293325804e-08, 'weight_decay': 2.7865853392804784e-07, 'alpha': 0.05352320846821692}. Best is trial 10 with value: 0.6708158850669861.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | conv1          | Conv2d      | 640   
5  | bn1            | BatchNorm2d | 128   
6  | dropout1       | Dropout     | 0     
7  | conv2          | Conv2d      | 73.9 K
8  | bn2            | BatchNorm2d | 256   
9  | dropout2       | Dropout     | 0     
10 | conv3          | Conv2d      | 147 K 
11 | bn3            | BatchNorm2d | 256   
12 | dropout3       | Dropout     | 0     
13 | conv4          | Conv2d      | 147 K 
14 | bn4            | BatchNorm2d | 256   
15 | dropout4       | Dropout     | 0     
16 | rnn            | GRU         | 21.9 K
17 | dropout_rnn    | Dropout     |

[I 2020-12-18 13:48:27,400] Trial 16 finished with value: 0.45098379254341125 and parameters: {'learning_rate': 2.9754947164857963e-06, 'weight_decay': 2.4978471804018545e-06, 'alpha': 0.05615303191338519}. Best is trial 10 with value: 0.6708158850669861.


In [8]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 17
Best trial:
  Value: 0.6708158850669861
  Params: 
    learning_rate: 9.521089031949996e-06
    weight_decay: 1.131463918804116e-06
    alpha: 2.3485658333624597


In [8]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 50
Best trial:
  Value: 0.9744158387184143
  Params: 
    learning_rate: 0.00017438033988242943
    weight_decay: 6.944923420383138e-05
    alpha: 1.7231602078617747


In [8]:
shutil.rmtree("checkpoints")

In [1]:
print("Number of finished trials: 50")

print("Best trial:")

print("0.73614310462657")

print("  Params: ")
print("  learning_rate:  0.0004881731392294949")
print("  weight_decay: 2.628328526113425e-08")
print("  alpha: 0.7608399210841633")


Number of finished trials: 50
Best trial:
0.73614310462657
  Params: 
  learning_rate:  0.0004881731392294949
  weight_decay: 2.628328526113425e-08
  alpha: 0.7608399210841633
